In [107]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [172]:
custom_stopwords = ['echo', 'alexa', "alexia", 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', "sony", 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def negation_handler(sentence):	

    """Handle negations using WordNet. See https://github.com/UtkarshRedd/Negation_handling for a clear explenation."""

    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [173]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
# dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [174]:
#  CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


# dataset = create_balanced_dataset(dataset)

In [175]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v).
    This was done in order to have compatibility with the wordnet lemmatizer.
    For example the pos "JJ" is transformed in "a".
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza.
    Ritorna sia il la lista tokenizzata ma come stringa sia come lista di tokens, dunque ritorna due elementi"""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    pos_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review) # elimina i numeri
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = negation_handler(tokens)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        sent_tokenized_reviews.append(clean_tokens)
        tokenized_reviews.append(detokenizer.detokenize(clean_tokens))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [176]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 

total number of tokens extracted are: 3171


In [177]:
# Elimino token con freq minore di 5
tot_tokens = []

for sent in new_sent_tok:
    for tok in sent:
        tot_tokens.append(tok)

freqs = nltk.FreqDist(tot_tokens)
cleaned_reviews = []

for sent in new_sent_tok:
    clean_sent = []
    for tok in sent:
        if freqs[tok] > 4:
            clean_sent.append(tok)
    cleaned_reviews.append(clean_sent)

cleaned_reviews

[['love'],
 ['love'],
 ['sometimes',
  'play',
  'game',
  'answer',
  'question',
  'correctly',
  'say',
  'get',
  'wrong',
  'answer',
  'like',
  'able',
  'turn',
  'light',
  'away',
  'home'],
 ['lot',
  'fun',
  'thing',
  'old',
  'control',
  'light',
  'play',
  'game',
  'like',
  'nice',
  'sound',
  'play',
  'music',
  'well'],
 ['music'],
 ['receive',
  'gift',
  'need',
  'another',
  'bluetooth',
  'something',
  'play',
  'music',
  'easily',
  'find',
  'smart',
  'speaker',
  'wait',
  'see',
  'else'],
 ['think',
  'one',
  "'ve",
  'purchase',
  'work',
  'get',
  'one',
  'every',
  'room',
  'house',
  'really',
  'like',
  'feature',
  'offer',
  'play',
  'music',
  'control',
  'light',
  'throughout',
  'house'],
 ['look', 'great'],
 ['love',
  'listen',
  'song',
  'heard',
  'since',
  'get',
  'news',
  'weather',
  'information',
  'great'],
 ['send', 'year', 'old', 'dad', 'talk', 'constantly'],
 ['love',
  'learn',
  'know',
  'thing',
  'still',
  'f

In [178]:
bigrams = Phrases(cleaned_reviews) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][12]

['love', 'love', 'love']

In [179]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], dataset["feedback"].values, test_size=0.20, random_state=10)

In [180]:
# Questa parte estrare le parole più rilevanti per ogni categoria, ma non la uso alla fine.
# Lo riprenderò più avanti
# FARE IL K-SQUARED

neg_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg):
    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p_category = pos_freq[w] / total_tokens_freq[w]
        p = pos_freq[w] / len(pos)
        data_dict["pos"][w] = p_category * p
    
    for w in neg:
        p_category = neg_freq[w] / total_tokens_freq[w]
        p = neg_freq[w] / len(neg)
        data_dict["neg"][w] = p_category * p

    risultato = {}

    for label in ["pos", "neg"]:
        values_list = list(data_dict[label].values())
        min_values = min(values_list)
        max_values = max(values_list)
        for k in data_dict[label]:
            val = data_dict[label][k]
            data_dict[label][k] = (val- min_values)/(max_values-min_values)

    return data_dict

data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized)

res = {}

for w in pos_tokenized+neg_tokenized:
    res[w] = max(data_dict["pos"].get(w, 0.01), data_dict["neg"].get(w, 0.01))

{k: v for k, v in sorted(res.items(), key = lambda item: item[1], reverse=True)}

{'love': 1.0,
 'not': 1.0,
 'device': 0.6981772976380158,
 'great': 0.6392804635456335,
 'would': 0.6100360961809296,
 'try': 0.5524876088841691,
 'stop': 0.5235527229644156,
 'use': 0.5180032583857497,
 'return': 0.5043440197503195,
 'screen': 0.4170743237731959,
 'work': 0.3982875886379849,
 'like': 0.37209037484151,
 'unplug': 0.3354954887005383,
 'time': 0.33356313683210465,
 'back': 0.3271000764566152,
 'get': 0.3222137012040661,
 'repair': 0.30527200462241516,
 'terrible': 0.29435796870531517,
 'buy': 0.28478034534949265,
 'good': 0.27651256841387667,
 'month': 0.2730044201718586,
 'one': 0.2647136680373493,
 'send_back': 0.2616158609540151,
 'sell': 0.25367838028703327,
 'idle': 0.2512215410329675,
 'sound': 0.24125107563670303,
 'music': 0.2394544669199159,
 'item': 0.23251176517508174,
 'disappointed': 0.22944817614572033,
 'product': 0.22892972261767452,
 'horrible': 0.21795971728561503,
 'disappointing': 0.21795971728561503,
 'thing': 0.216688179120516,
 'want': 0.2155117092

In [181]:
# tfidf weighting sui token, per ora senza pos. Mi sono reso conto che la pos per averla più accurata la devo fare prima del pre processing.

tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 10
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

love 0.35215964443583386
think 0.6934931377328509
would 0.6285307094711221


In [182]:
w2v_model = Word2Vec(X_train, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007)

In [183]:
w2v_model.train(X_train, total_examples=len(X_train), epochs=100)

(1735932, 2256700)

In [184]:
w2v_model.wv.most_similar("love", topn = 10)

[('bought', 0.21078026294708252),
 ('sense', 0.18446293473243713),
 ('absolutely', 0.18154695630073547),
 ('one', 0.17799833416938782),
 ('mom', 0.17721320688724518),
 ('listen_music', 0.17448307573795319),
 ('chat', 0.17325447499752045),
 ('super_easy', 0.17266851663589478),
 ('arlo', 0.17036005854606628),
 ('improvement', 0.16968993842601776)]

In [185]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('bass', -0.02588074840605259),
 ('properly', -0.026854217052459717),
 ('might', -0.02711598202586174),
 ('fill', -0.027565956115722656),
 ('snooze', -0.028415128588676453),
 ('provide', -0.03195636719465256),
 ('understand', -0.03862810134887695),
 ('software', -0.03903976455330849),
 ('case', -0.05645255744457245),
 ('playlist', -0.06353271007537842)]

In [186]:
df = pd.DataFrame(tfidf_model.toarray(), columns = tfidf.get_feature_names_out())
df

,'ll,'re,'ve,....,.....,......,ability,able,absolutely,accent,...,worth_money,would,would_recommend,wrong,yard,year,year_old,yell,yes,yet
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.249588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.222630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
g_model = []
# google_model = api.load('word2vec-google-news-300', True)
# g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [188]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False, test = False):

    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * perso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    if test:
        for word in tokens:
            try:
                if pretrained:
                    vec += g_model[word] * res[word]
                else:
                    vec += w2v_model.wv[word] * res[word]
                count +=1
            except KeyError:
                print("non trovo", word)
                continue

        if count!= 0:
            vec = vec / norm(vec)
        
        return vec

    if len(tokens) != len(weights):
        print("nope")

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += g_model[word] * res[word] * weight
            else:
                vec += w2v_model.wv[word] * res[word] * weight
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [189]:
# qui viene creato per ogni recensione il vettore

w2v_X_train = np.zeros((len(X_train), 300))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_X_train)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.018290,-0.018557,-0.033033,0.003662,0.014204,-0.042024,0.028271,0.061715,0.135896,-0.000744,...,0.055315,0.121493,-0.010648,0.001769,-0.021777,0.000086,-0.012158,-0.064747,-0.098045,-0.108662
1,0.018904,-0.049392,0.018222,0.075438,0.070576,-0.006408,-0.000153,-0.052854,0.152767,-0.000599,...,0.091731,0.041171,-0.022804,0.029457,0.048844,0.085374,0.000503,-0.032044,0.001499,-0.073479
2,-0.040300,0.011389,-0.034307,0.000448,0.061836,-0.021893,0.014222,0.060346,0.113635,-0.004765,...,0.022901,0.078537,-0.003613,0.009377,-0.025228,-0.005499,-0.050138,-0.015860,-0.008331,-0.095167
3,-0.034335,0.121544,0.034843,0.013819,0.021163,0.047382,0.096808,0.000038,0.027673,0.021880,...,-0.103949,-0.054832,0.047844,0.016787,0.038083,-0.077889,0.115987,0.023756,0.118006,0.017428
4,-0.014079,0.023996,0.021399,0.007115,0.008356,-0.025508,0.005812,0.138480,0.122211,-0.032325,...,0.006663,0.046139,-0.047649,0.056522,-0.027049,0.016254,-0.078306,0.006615,-0.012946,-0.123034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,0.082512,-0.066278,0.009199,0.045618,-0.023292,-0.029989,-0.006650,0.048763,0.009670,0.019241,...,-0.011732,-0.085148,-0.019416,0.002567,0.020140,-0.003354,-0.043775,0.006157,-0.021230,-0.036745
2394,-0.007475,-0.048398,0.035438,0.028949,-0.079039,0.001247,-0.035675,0.151994,0.057641,0.058481,...,-0.012705,0.033346,-0.022989,0.073153,-0.001653,-0.006718,-0.049216,0.017678,-0.078795,-0.009252
2395,-0.024378,-0.026479,-0.056882,0.034012,-0.008947,-0.062606,0.076620,-0.025199,0.001328,-0.084722,...,-0.014409,-0.060073,-0.004314,0.019655,0.029670,0.094956,-0.016765,0.051429,-0.040007,-0.059116
2396,-0.041030,-0.008750,-0.017700,-0.011469,-0.025703,-0.063599,0.002138,0.055585,0.107449,0.034702,...,0.015823,0.108159,0.019756,0.002585,-0.028823,-0.032597,-0.022463,-0.024098,-0.106879,-0.079212


In [190]:
X_test_vec = np.zeros((len(X_test), 300))
for i in range(len(X_test)):
    X_test_vec[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], 300, X_test_tf[i,:].data, False)

X_test_vec[0]

array([-3.86537363e-02, -5.43494256e-02,  4.22453532e-02, -4.35949879e-02,
        6.10820670e-02, -5.65659185e-02, -2.85846127e-02, -3.14168387e-02,
        1.40639014e-01,  2.27680843e-02,  3.98369052e-02,  5.06256443e-02,
        6.46252712e-02,  6.62833382e-02, -2.25755500e-02, -7.96714789e-02,
       -3.44036664e-02, -1.10134133e-02,  2.10436390e-02, -1.72540613e-02,
       -4.30186883e-02,  7.13505979e-02, -7.35387711e-02, -3.52532074e-03,
       -4.69208821e-02,  4.95348759e-02,  2.49735750e-02, -8.32272122e-02,
        5.25610455e-02, -5.20042760e-02,  3.36418242e-02, -5.63481777e-02,
        8.70948741e-02,  3.28536736e-02,  9.78267444e-02, -8.37611358e-02,
       -1.74886155e-02, -8.90046419e-02,  1.60996468e-02,  1.59960245e-02,
       -3.18002709e-03, -5.00556999e-02, -9.40041383e-02, -1.68242575e-03,
       -3.15282713e-02,  5.51912198e-02, -5.22589179e-02,  2.81910717e-02,
        2.66804641e-02,  4.13721185e-02,  4.56137015e-02,  5.94560688e-02,
       -5.33920634e-02, -

In [191]:
svm = SVC(max_iter=3000, class_weight = "balanced")
svm_model = svm.fit(w2v_X_train, Y_train)
predictions = svm_model.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.81      0.63        48
           1       0.98      0.93      0.96       552

    accuracy                           0.92       600
   macro avg       0.75      0.87      0.79       600
weighted avg       0.95      0.92      0.93       600



In [129]:
# risultati migliori:

# w2v + tf-idf weights + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better

In [192]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [193]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

In [194]:
grid = GridSearchCV(SVC(), param_grid, refit ='f1_macro', scoring="f1_macro", cv=kfold, verbose=3)
grid.fit(w2v_X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.687 total time=   1.0s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.680 total time=   0.8s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.708 total time=   0.8s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.733 total time=   0.7s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.682 total time=   0.7s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.684 total time=   0.4s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.634 total time=   0.4s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.665 total time=   0.4s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.701 total time=   0.5s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             refit='f1_macro', scoring='f1_macro', verbose=3)

In [195]:
print(grid.best_params_)

{'C': 10, 'class_weight': None, 'gamma': 1, 'kernel': 'rbf'}


In [196]:
grid_predictions = grid.predict(X_test_vec)

print(classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.77      0.69      0.73        48
           1       0.97      0.98      0.98       552

    accuracy                           0.96       600
   macro avg       0.87      0.83      0.85       600
weighted avg       0.96      0.96      0.96       600



In [138]:
df_test = pd.read_csv("scraped_data/echo_dot.csv", sep = "\t")
df_test

,variation,verified_reviews,rating,date,feedback
0,Echo Dot,I hate this little device from hell. I used to...,1,"August 4, 2020",0
1,Echo Dot,Most of the features that sounded so great cos...,1,"December 21, 2019",0
2,Echo Dot,I purchased multiple Blink items overall revie...,3,"March 15, 2022",1
3,Echo Dot,I've been an Echo user since the first tall Ec...,2,"January 6, 2022",0
4,Echo Dot,I am having problems with the GE lights and Al...,3,"October 14, 2022",1
...,...,...,...,...,...
485,Echo Dot,this device is kind of lost on me. I don't rea...,3,"February 3, 2021",1
486,Echo Dot,They didn't have any Jamaican or Rasta voices,3,"April 8, 2022",1
487,Echo Dot,I have newer Alexa's but I ordered this one be...,2,"June 8, 2022",0
488,Echo Dot,Having issues with voice recognition. Purchase...,2,"January 16, 2020",0
